In [1]:
from os import sys
# Path to workspace
sys.path.insert(0, '/workspace/dense-self-supervised-representation-learning-for-3D-shapes/')

import h5py
import torch
import numpy as np
from tqdm import tqdm
import k3d

In [2]:
import neptune.new as neptune
from workspace.utils.train_loop import *

params = {
    'name': 'coseg_sem',
    'dataset': 'modelnet',
    'batch_size': 10,
    'tau': 0.07,
    'n_output': 512,
    'result_dim': 3,
    'hidden_dim': 256,
    'total_epochs': 100,
    'lr': 0.001,
    'weight_decay': 1e-5,
    'save_every': 50,
    'weights_root': 'weights/'
}

tags = ['coseg', 'segmentation']

logger = neptune.init(
    project="igor3661/crossmodal",
    name=params['name'],
    tags=tags,
    api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcG'\
              'lfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiN'\
              'zcxMGNkOS04ZjU3LTRmNDMtOWFjMS1kNDNkZDZlNDI4YWYifQ==',
)  # your credentials


logger['parameters'] = params

device = 'cuda:2'

/opt/conda/lib/python3.8/site-packages/neptune/new/internal/utils/git.py:35: UserWarning: GitPython could not be initialized
  warnings.warn("GitPython could not be initialized")
/opt/conda/lib/python3.8/site-packages/neptune/new/internal/utils/git.py:35: UserWarning: GitPython could not be initialized
  warnings.warn("GitPython could not be initialized")


https://app.neptune.ai/igor3661/crossmodal/e/CROSS-214
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [3]:
from torch.utils.data import Dataset, DataLoader
from workspace.crossmodal.utils.meshnet_preprop import *


class PointSegDataset(Dataset):
    def __init__(self, data_path, indexes=None, transform=None):
        super().__init__()
        self.transform = transform
        self.indexes = indexes
        self.file = h5py.File(data_path, 'r')

    def __getitem__(self, index):  
        index = self.indexes[index]
        points = self.file['points'][index][:]
        labels = self.file['points_labels'][index]

        if self.transform is not None:
            points = self.transform(points)

        points = torch.from_numpy(points).float()
        points = torch.permute(points, (1, 0))
        return points, labels

        
    def __len__(self):
        return len(self.indexes)
    
    
class MeshnetSegDataset(Dataset):
    def __init__(self, data_path, indexes=None, rotation=None, jitter=None):
        super().__init__()
        self.rotation = rotation
        self.indexes = indexes
        self.jitter = jitter
        self.file = h5py.File(data_path, 'r')

    def __getitem__(self, index):
        index = self.indexes[index]
        faces = self.file['faces'][index][:].reshape(-1, 3)
        vertices = self.file['vertices'][index][:].reshape(-1, 3)
        labels = self.file['faces_labels'][index]
        
        
        if self.rotation is not None:
            vertices = self.rotation(vertices)
        
        features, neighbors = process_mesh(faces, vertices)
        

        features = torch.from_numpy(features).float()
        neighbors = torch.from_numpy(neighbors).long()
        labels = torch.from_numpy(labels).long()

        features = torch.permute(features, (1, 0))
        centers, corners, normals = features[:3], features[3:12], features[12:]
        
        if self.jitter is not None:
            centers = self.jitter(centers).float()
        
        corners = corners - torch.cat([centers, centers, centers], 0).float()
        
        num_faces = neighbors.shape[0]
        if num_faces < 500:
            fill_idx = np.random.choice(num_faces, 500 - num_faces)
            centers = (torch.cat([centers, centers[:, fill_idx]], dim=1))
            corners = (torch.cat([corners, corners[:, fill_idx]], dim=1))
            normals = (torch.cat([normals, normals[:, fill_idx]], dim=1))
            neighbors = (torch.cat([neighbors, neighbors[fill_idx]]))
            labels = (torch.cat([labels, labels[fill_idx]]))

        return centers, corners, normals, neighbors, labels
        
    def __len__(self):
        return len(self.indexes)

In [4]:
DATASET = 'chairs.h5'

with h5py.File(DATASET, 'r') as h5:
    N_CLASSES = len(np.unique(h5['points_labels'][:]))
    LENGTH = h5['points_labels'].shape[0]
    split = np.random.permutation(LENGTH)
    train_indexes = split[:int(0.05 * LENGTH) + 1]
    test_indexes = split[int(0.05 * LENGTH) + 1:]


In [5]:
train_data = MeshnetSegDataset(DATASET, indexes=train_indexes)
test_data = MeshnetSegDataset(DATASET, indexes=test_indexes)


train_loader = DataLoader(
    train_data,
    shuffle=True,
    batch_size=params['batch_size'],
    num_workers=10,
)
test_loader = DataLoader(
    test_data,
    shuffle=False,
    batch_size=params['batch_size'],
    num_workers=10
)

In [6]:
class Transpose(torch.nn.Module):
    def __init__(self, *dims):
        super().__init__()
        self.dims = dims

    def forward(self, data):
        return data.transpose(*self.dims)
    

class NormalsModel(torch.nn.Module):
    def __init__(self, model, model_output_dim, result_dim, hidden_dim):
        super().__init__()
        self.model = model
        self.head = torch.nn.Sequential(
            Transpose(1, 2),
            torch.nn.Linear(model_output_dim, hidden_dim),
            Transpose(1, 2),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            Transpose(1, 2),
            torch.nn.Linear(hidden_dim, 128),
            Transpose(1, 2),
        )
        
    def forward(self, data):
        return self.head(self.model.forward_features(data))

In [7]:
from workspace.models.dgcnn import DGCNN
from workspace.models.meshnet import MeshNet

meshnet = MeshNet(n_patches=5)

model = NormalsModel(
    meshnet,
    model_output_dim=params['n_output'],
    hidden_dim=params['hidden_dim'],
    result_dim=params['result_dim']
).to(device).eval()

model.load_state_dict(torch.load('weights/CROSS-64/100epoch.pt'))

<All keys matched successfully>

In [8]:
head = torch.nn.Sequential(
    Transpose(1, 2),
    torch.nn.Linear(params['n_output'], params['n_output'] // 2),
    Transpose(1, 2),
    torch.nn.BatchNorm1d(params['n_output'] // 2),
    torch.nn.ReLU(),
    Transpose(1, 2),
    torch.nn.Linear(params['n_output'] // 2, N_CLASSES),
    Transpose(1, 2),
)

model.head = head.to(device)

In [9]:
class Projector(torch.nn.Module):
    def __init__(self, model_output_dim):
        super().__init__()
        self.head = torch.nn.Sequential(
                Transpose(1, 2),
                torch.nn.Linear(model_output_dim + 3, model_output_dim * 2),
                Transpose(1, 2),
                torch.nn.BatchNorm1d(model_output_dim * 2),
                torch.nn.ReLU(),
                Transpose(1, 2),
                torch.nn.Linear(model_output_dim * 2, model_output_dim),
                Transpose(1, 2),
            )
    
    def forward(self, data):
        return self.head(data)
    
proj = Projector(params['n_output']).to(device)
proj.load_state_dict(torch.load('weights/CROSS-74/100epoch.pt'))
model.proj = proj

In [10]:
import torch.nn.functional as F


def move_to_device(data, device='cpu'):
    if isinstance(data, list):
        return [item.to(device) for item in data]
    else:
        return data.to(device)


def forward_clouds( 
    model,
    batch, # raw data from dataloader
    logger, # neptune run
    mode # 'train'/'val'
): # -> loss

    batch = move_to_device(batch, device)
    
    points = batch[0]
    labels = batch[1]
        
    out = model(points)
        
    loss = F.cross_entropy(out, labels.long())
    
    
    return {
        'loss': loss
    }

def forward_meshes( 
    model,
    batch, # raw data from dataloader
    logger, # neptune run
    mode # 'train'/'val'
): # -> loss
    
    batch = move_to_device(batch, device)
    
    meshes = batch[0:4]
    labels = batch[4]
        
    out = model(meshes)
        
    loss = F.cross_entropy(out, labels.long())
    
    
    return {
        'loss': loss
    }

def forward_proj(
    model,
    batch, # raw data from dataloader
    logger, # neptune run
    mode
):
    batch = move_to_device(batch, device)
    
    meshes = batch[0:4]
    centers = meshes[0]
    labels = batch[4]
        
    out = model.model.forward_features(meshes)
    
    out = torch.cat([out, centers], dim=1)
    
    out = model.head(model.proj(out))
        
    loss = F.cross_entropy(out, labels.long())
    
    
    return {
        'loss': loss
    }

In [11]:
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=params['lr'],
    weight_decay=params['weight_decay']
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_loader) * params['total_epochs'])

In [12]:
train_model(model, params, logger,  train_loader, test_loader, optimizer, scheduler, forward_proj)

Validation: 100%|██████████| 38/38 [00:05<00:00,  7.52it/s, Loss=0.536]


Validation: 100%|██████████| 38/38 [00:03<00:00, 12.00it/s, Loss=0.54] 


In [13]:
@torch.no_grad()
def get_preds_targets_clouds(model, loader):
    model.eval()
    preds = []
    targets = []
    for points, labels in tqdm(loader):
        out = model(points.to(device)).cpu()
        preds.append(torch.max(out, dim=1).indices)
        targets.append(labels)
        
    preds = torch.cat(preds)
    targets = torch.cat(targets)
    return preds, targets

@torch.no_grad()
def get_preds_targets_meshes(model, loader):
    model.eval()
    preds = []
    targets = []
    for centers, corners, normals, neighbors, labels in tqdm(loader):
        out = model([centers.to(device), corners.to(device), normals.to(device), neighbors.to(device)]).cpu()
        preds.append(torch.max(out, dim=1).indices)
        targets.append(labels)
        
    preds = torch.cat(preds)
    targets = torch.cat(targets)
    return preds, targets

@torch.no_grad()
def get_preds_targets_proj(model, loader):
    model.eval()
    preds = []
    targets = []
    for centers, corners, normals, neighbors, labels in tqdm(loader):
        centers = centers.to(device)
        corners = corners.to(device)
        normals = normals.to(device)
        neighbors = neighbors.to(device)
        out = model.model.forward_features([centers, corners, normals, neighbors])
        out = torch.cat([out, centers], dim=1)
        out = model.head(model.proj(out))
        
        preds.append(torch.max(out, dim=1).indices)
        targets.append(labels)
        
    preds = torch.cat(preds)
    targets = torch.cat(targets)
    return preds, targets

In [14]:
p, l = get_preds_targets_proj(model, test_loader)

100%|██████████| 38/38 [00:02<00:00, 12.90it/s]


In [15]:
def mIOU(label, pred, num_classes=N_CLASSES):
    iou_list = list()
    present_iou_list = list()

    pred = pred.view(-1)
    label = label.view(-1)
    # Note: Following for loop goes from 0 to (num_classes-1)
    # and ignore_index is num_classes, thus ignore_index is
    # not considered in computation of IoU.
    for sem_class in range(num_classes):
        pred_inds = (pred == sem_class)
        target_inds = (label == sem_class)
        if target_inds.long().sum().item() == 0:
            iou_now = float('nan')
        else: 
            intersection_now = (pred_inds[target_inds]).long().sum().item()
            union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
            iou_now = float(intersection_now) / float(union_now)
            present_iou_list.append(iou_now)
        iou_list.append(iou_now)
    return np.mean(present_iou_list)

In [16]:
mIOU(l, p)

0.7627959185143064

In [17]:
# N_PARTS = 4
# N_CLASSES = 1
# NUM_POINT = 1024

# seg_classes = {'Aliens': [0, 1, 2, 3]}
# seg_label_to_cat = {0: 'Aliens'}

# @torch.no_grad()
# def calc_metrics(model, loader, device):
#     model.eval()
#     test_metrics = {}
#     total_correct = 0
#     total_seen = 0
#     total_seen_class = [0 for _ in range(N_PARTS)]
#     total_correct_class = [0 for _ in range(N_PARTS)]
#     shape_ious = {cat: [] for cat in seg_classes.keys()}

#     for batch_id, (points, label) in tqdm(enumerate(loader), total=len(loader)):
#         target = torch.zeros(points.shape[0])
#         cur_batch_size, _, NUM_POINT = points.size()
#         points, label, target = points.to(device), label.long().to(device), target.long()

#         seg_pred = model(points.to(device)).transpose(2, 1)

#         cur_pred_val = seg_pred.cpu().data.numpy()
#         cur_pred_val_logits = cur_pred_val
#         cur_pred_val = np.zeros((cur_batch_size, NUM_POINT)).astype(np.int32)
#         target = target.data.numpy()

#         for i in range(cur_batch_size):
#             cat = seg_label_to_cat[target[i, 0]]
#             logits = cur_pred_val_logits[i, :, :]
#             cur_pred_val[i, :] = np.argmax(logits[:, seg_classes[cat]], 1) + seg_classes[cat][0]

#         correct = np.sum(cur_pred_val == target)
#         total_correct += correct
#         total_seen += (cur_batch_size * NUM_POINT)

#         for l in range(N_PARTS):
#             total_seen_class[l] += np.sum(target == l)
#             total_correct_class[l] += (np.sum((cur_pred_val == l) & (target == l)))

#         for i in range(cur_batch_size):
#             segp = cur_pred_val[i, :]
#             segl = target[i, :]
#             cat = seg_label_to_cat[segl[0]]
#             part_ious = [0.0 for _ in range(len(seg_classes[cat]))]
#             for l in seg_classes[cat]:
#                 if (np.sum(segl == l) == 0) and (
#                         np.sum(segp == l) == 0):  # part is not present, no prediction as well
#                     part_ious[l - seg_classes[cat][0]] = 1.0
#                 else:
#                     part_ious[l - seg_classes[cat][0]] = np.sum((segl == l) & (segp == l)) / float(
#                         np.sum((segl == l) | (segp == l)))
#             shape_ious[cat].append(np.mean(part_ious))

#     all_shape_ious = []
#     for cat in shape_ious.keys():
#         for iou in shape_ious[cat]:
#             all_shape_ious.append(iou)
#         shape_ious[cat] = np.mean(shape_ious[cat])

#     mean_shape_ious = np.mean(list(shape_ious.values()))
#     test_metrics['accuracy'] = total_correct / float(total_seen)
#     test_metrics['class_avg_accuracy'] = np.mean(
#         np.array(total_correct_class) / np.array(total_seen_class, dtype=float))

#     for cat in sorted(shape_ious.keys()):
#         print('eval mIoU of %s %f' % (cat + ' ' * (14 - len(cat)), shape_ious[cat]))

#     test_metrics['class_avg_iou'] = mean_shape_ious
#     test_metrics['instance_avg_iou'] = np.mean(all_shape_ious)

#     print('Accuracy is: %.5f' % test_metrics['accuracy'])
#     print('Class avg accuracy is: %.5f' % test_metrics['class_avg_accuracy'])
#     print('Class avg mIOU is: %.5f' % test_metrics['class_avg_iou'])
#     print('Instance avg mIOU is: %.5f' % test_metrics['instance_avg_iou'])

#     return test_metrics